In [ ]:
# final_phonebridge_logger.py
import os, csv, time
from datetime import datetime
from pathlib import Path
from dotenv import load_dotenv
from arduino_iot_cloud import ArduinoCloudClient

# 1) Load Python device creds (Alysia)
load_dotenv("Pass.env")  # change to ".env" if that's your file name
DEVICE_ID     = os.getenv("DEVICE_ID")
DEVICE_SECRET = os.getenv("DEVICE_SECRET")
if not DEVICE_ID or not DEVICE_SECRET:
    raise SystemExit("Missing DEVICE_ID or DEVICE_SECRET in Pass.env")

# 2) Cloud variable names on PhoneBridge (linked to iPhone Thing)
VAR_X = "var_x"
VAR_Y = "var_y"
VAR_Z = "var_z"   # exact case!

# 3) Output files
DATA_DIR = Path("data"); DATA_DIR.mkdir(exist_ok=True)
FX = DATA_DIR / "accelerometer_x.csv"
FY = DATA_DIR / "accelerometer_y.csv"
FZ = DATA_DIR / "accelerometer_z.csv"
FALL = DATA_DIR / "accelerometer_xyz.csv"

def ensure_header(p, cols):
    if not p.exists():
        with p.open("w", newline="") as f:
            csv.writer(f).writerow(cols)

ensure_header(FX,   ["timestamp","value"])
ensure_header(FY,   ["timestamp","value"])
ensure_header(FZ,   ["timestamp","value"])
ensure_header(FALL, ["timestamp","x","y","z"])

# 4) Connect Python device to PhoneBridge
client = ArduinoCloudClient(
    device_id=DEVICE_ID,
    username=DEVICE_ID,
    password=DEVICE_SECRET,
    sync_mode=True            # we'll poll with client.update()
)

# Register the three PhoneBridge variables (must match exactly)
client.register(VAR_X, value=None)
client.register(VAR_Y, value=None)
client.register(VAR_Z, value=None)

client.start()
print("Connected. Polling cloud and logging once per second… (Ctrl+C to stop)")
print("Reading:", VAR_X, VAR_Y, VAR_Z)

# Keep the freshest values we see while polling
latest_x = latest_y = latest_z = None

try:
    while True:
        # Poll ~20 times in this second to catch fresh MQTT updates
        t_end = time.time() + 1.0
        while time.time() < t_end:
            client.update()
            x = client.get(VAR_X)
            y = client.get(VAR_Y)
            z = client.get(VAR_Z)
            if x is not None: latest_x = x
            if y is not None: latest_y = y
            if z is not None: latest_z = z
            time.sleep(0.03)  # ~33 Hz polling

        # Write one row per second (re-using last seen values if any are None)
        ts = datetime.now().isoformat(timespec="seconds")
        with FX.open("a", newline="") as f: csv.writer(f).writerow([ts, latest_x])
        with FY.open("a", newline="") as f: csv.writer(f).writerow([ts, latest_y])
        with FZ.open("a", newline="") as f: csv.writer(f).writerow([ts, latest_z])
        with FALL.open("a", newline="") as f: csv.writer(f).writerow([ts, latest_x, latest_y, latest_z])

        print(f"[{ts}] X={latest_x}, Y={latest_y}, Z={latest_z}")

except KeyboardInterrupt:
    print("Stopped.")


Connected. Polling cloud and logging once per second… (Ctrl+C to stop)
Reading: var_x var_y var_z
[2025-09-16T14:21:32] X=-0.0284881591796875, Y=-0.4065399169921875, Z=-0.9037322998046875
[2025-09-16T14:21:33] X=-0.0284881591796875, Y=-0.4065399169921875, Z=-0.9037322998046875
[2025-09-16T14:21:34] X=-0.0284881591796875, Y=-0.4065399169921875, Z=-0.9037322998046875
[2025-09-16T14:21:35] X=-0.0284881591796875, Y=-0.4065399169921875, Z=-0.9037322998046875
[2025-09-16T14:21:36] X=-0.0284881591796875, Y=-0.4065399169921875, Z=-0.9037322998046875
[2025-09-16T14:21:37] X=-0.0284881591796875, Y=-0.4065399169921875, Z=-0.9037322998046875
[2025-09-16T14:21:38] X=-0.0284881591796875, Y=-0.4065399169921875, Z=-0.9037322998046875
[2025-09-16T14:21:39] X=-0.0284881591796875, Y=-0.4065399169921875, Z=-0.9037322998046875
[2025-09-16T14:21:40] X=-0.0284881591796875, Y=-0.4065399169921875, Z=-0.9037322998046875
[2025-09-16T14:21:42] X=-0.0284881591796875, Y=-0.4065399169921875, Z=-0.9037322998046875
[2

In [2]:
# creating only one file istead of creating 3 different csv files 

import os, csv, time
from datetime import datetime
from pathlib import Path
from dotenv import load_dotenv
from arduino_iot_cloud import ArduinoCloudClient

# 1) Load Python device creds (Alysia)
load_dotenv("Pass.env")  # change to ".env" if that's your filename
DEVICE_ID     = os.getenv("DEVICE_ID")
DEVICE_SECRET = os.getenv("DEVICE_SECRET")
if not DEVICE_ID or not DEVICE_SECRET:
    raise SystemExit("Missing DEVICE_ID/DEVICE_SECRET in env file")

# 2) PhoneBridge variable names (linked to iPhone Thing)
VAR_X = "var_x"
VAR_Y = "var_y"
VAR_Z = "var_z"

# 3) Output file
DATA_DIR = Path("data"); DATA_DIR.mkdir(exist_ok=True)
OUT = DATA_DIR / "accelerometer_xyz.csv"
if not OUT.exists():
    with OUT.open("w", newline="") as f:
        csv.writer(f).writerow(["timestamp", "x", "y", "z"])

# 4) Connect client (polling mode)
client = ArduinoCloudClient(
    device_id=DEVICE_ID,
    username=DEVICE_ID,
    password=DEVICE_SECRET,
    sync_mode=True
)

client.register(VAR_X, value=None)
client.register(VAR_Y, value=None)
client.register(VAR_Z, value=None)

client.start()
print("Connected. Logging <timestamp,x,y,z> once per second… (Ctrl+C to stop)")

latest_x = latest_y = latest_z = None

try:
    while True:
        # Poll fast for fresh values during this 1-second window
        end = time.time() + 1.0
        while time.time() < end:
            client.update()
            x = client.get(VAR_X);  y = client.get(VAR_Y);  z = client.get(VAR_Z)
            if x is not None: latest_x = x
            if y is not None: latest_y = y
            if z is not None: latest_z = z
            time.sleep(0.03)  # ~33 Hz polling

        # Write one row per second (reuse last seen values if any are None)
        ts = datetime.now().isoformat(timespec="seconds")
        with OUT.open("a", newline="") as f:
            csv.writer(f).writerow([ts, latest_x, latest_y, latest_z])
        print(f"[{ts}] {latest_x}, {latest_y}, {latest_z}")

except KeyboardInterrupt:
    print("Stopped.")


Connected. Logging <timestamp,x,y,z> once per second… (Ctrl+C to stop)
[2025-09-16T14:30:18] -0.0503387451171875, -0.394195556640625, -0.9127044677734375
[2025-09-16T14:30:19] -0.0503387451171875, -0.394195556640625, -0.9127044677734375
[2025-09-16T14:30:20] -0.0503387451171875, -0.394195556640625, -0.9127044677734375
[2025-09-16T14:30:21] -0.0503387451171875, -0.394195556640625, -0.9127044677734375
[2025-09-16T14:30:22] -0.0503387451171875, -0.394195556640625, -0.9127044677734375
[2025-09-16T14:30:23] -0.0503387451171875, -0.394195556640625, -0.9127044677734375
[2025-09-16T14:30:24] -0.0503387451171875, -0.394195556640625, -0.9127044677734375
[2025-09-16T14:30:25] -0.0503387451171875, -0.394195556640625, -0.9127044677734375
[2025-09-16T14:30:26] -0.0503387451171875, -0.394195556640625, -0.9127044677734375
[2025-09-16T14:30:27] -0.0503387451171875, -0.394195556640625, -0.9127044677734375
[2025-09-16T14:30:28] -0.0503387451171875, -0.394195556640625, -0.9127044677734375
[2025-09-16T14:3